# DERBi PIE Semantics Module

In [5]:
import numpy as np

In [6]:
from scipy import spatial

In [10]:
import gutenbergpy.textget
def getbook(book, outfile):
  """
  Download a book from project Gutenberg and save it 
  to the specified outfile
  """
  print(f"Downloading Project Gutenberg ID {book}")
  raw_book = gutenbergpy.textget.get_text_by_id(book)
  clean_book = gutenbergpy.textget.strip_headers(raw_book)
  if not outfile:
    outfile = f'{book}.txt'
    print(f"Saving book as {outfile}")
  with open(outfile, 'wb') as file:
    file.write(clean_book)
    file.close()

In [20]:
# these are Project Gutenberg IDs and information
caesar_ids = [218, 18837, 29645]
caesar_files = [f"gen/books/caesar/{x}.txt" for x in caesar_ids]
caesar_titles = ["Bello Gallico I-IV", "Bello Gallico V-VII", "The Gate to Caesar"]
cicero_ids = [14970, 226, 47001]
cicero_files = [f"gen/books/cicero/{x}.txt" for x in cicero_ids]
cicero_titles = ["Academia", "Orations", "De Officiis"]

In [21]:
foo = [getbook(x, f"gen/books/caesar/{x}.txt") for x in caesar_ids]
foo = [getbook(x, f"gen/books/cicero/{x}.txt") for x in cicero_ids]

FileNotFoundError: [Errno 2] No such file or directory: 'gen/books/caesar/218.txt'